## Load Data

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
root_path = os.path.join(os.curdir, '../../data/book/processed')

raw_users = pd.read_csv(root_path + '/users.csv')
raw_books = pd.read_csv(root_path + '/books.csv')
raw_train_ratings = pd.read_csv(root_path + '/train_ratings.csv')
raw_test_ratings = pd.read_csv(root_path + '/test_ratings.csv')

In [4]:
raw_users.head()

,user_id,city,state,country,age_bin
0,8,timmins,ontario,canada,0
1,11400,ottawa,ontario,canada,4
2,11676,NaN,NaN,NaN,0
3,67544,toronto,ontario,canada,3
4,85526,victoria,britishcolumbia,canada,3


In [9]:
raw_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 149570 non-null  object
 1   book_author          149569 non-null  object
 2   year_of_publication  149570 non-null  int64 
 3   publisher            149570 non-null  object
 4   summary              149570 non-null  int64 
 5   major_cat            149570 non-null  object
 6   isbn_area            149570 non-null  object
dtypes: int64(2), object(5)
memory usage: 8.0+ MB


In [14]:
raw_books['isbn_area'].value_counts()

isbn_area
1       134405
3         6706
2         3405
84        3399
88         482
         ...  
9991         1
9960         1
9979         1
9974         1
9998         1
Name: count, Length: 68, dtype: int64

## users 전처리 함수
- location
    - 소문자 변환, 공백 및 특수문자 삭제
    - , 기준으로 앞에서부터 city, state, country 분리
    - state 결측치에 city 기준으로 최빈 state 대체
    - country 결측치에 city 기준으로 최빈 country 대체
    - 미국이면 state 사용, 그외는 country 사용
    - 결측치는 'na' 입력
- age -> age_bin
    - 결측치는 0 입력
    - 결측치, 20세 미만부터 70세 미만, 70세 이상 100세 미만 총 8개 구간으로 범주화
- city, state, country, age
    - 삭제
- (선택) 데이터가 적은 class는 others로 통합
    - location
    - 시간이 오래 걸림

In [4]:
def users_preprocess(raw_users) :
    users = raw_users.copy()
    
    # location
    users['location'] = users['location'].str.lower().replace('[^0-9a-zA-Z:,]', '', regex=True)
    users['city'] = users['location'].apply(lambda x: x.split(',')[-3].strip())
    users['state'] = users['location'].apply(lambda x: x.split(',')[-2].strip())
    users['country'] = users['location'].apply(lambda x: x.split(',')[-1].strip())
    users = users.replace('na', np.nan)
    users = users.replace('', np.nan)
    users.drop(columns=['location'], inplace=True)
    
    city_state_map = dict(users.groupby('city')['state']
                          .value_counts().sort_values().index.tolist())
    city_country_map = dict(users.groupby('city')['country']
                            .value_counts().sort_values().index.tolist())
    users['state'] = users['city'].map(city_state_map)
    users['country'] = users['city'].map(city_country_map)
    
    users['location'] = users['country'].copy()
    users['location'] = np.where(users['location']=='usa',
                             users['state'],
                             users['location'])
    users['location'].fillna('na', inplace=True)
    
    # age
    users['age'].fillna(0, inplace=True)
    bins = [0, 1, 20, 30, 40, 50, 60, 70, 100]
    users['age_bin'] = pd.cut(x=users['age'], bins=bins, right=False, labels=range(8))

    # 선택
    location_cnt = users['location'].value_counts()
    low_cnt_location = location_cnt[location_cnt < 10].index.tolist()
    for location in low_cnt_location :
        users['location'] = np.where(users['location']==location,
                                     'others', users['location'])
    users.drop(columns=['city', 'state', 'country', 'age'], inplace=True)
    
    return users

In [5]:
users = users_preprocess(raw_users)

In [6]:
users.head()

,user_id,location,age_bin
0,8,canada,0
1,11400,canada,4
2,11676,na,0
3,67544,canada,3
4,85526,canada,3


## books 전처리 함수
- isbn 
    - img_url 에서 파싱해서 대체
    - books, ratings 같이 처리
- book_author
    - 소문자 변환, 공백 및 특수문자 삭제
- year_of_publication
    - 1950년 이전 ~ 2010년 이전까지 7개 구간으로 범주화
- publisher
    - 소문자 변환, 공백 및 특수문자 삭제
- category -> major_cat
    - 소문자 변환, 공백 및 특수문자 삭제
    - 동일 저자에 대해 최빈 카테고리로 통일
        - ex) 저자가 stephenking인 모든 책의 카테고리는 fiction
    - 남은 결측치에 대해서 출판사의 최빈 카테고리 적용 (통일은 X)
    - 그래도 남은 결측치에는 'na'
    - 최종적으로 같은 카테고리로 분류 가능한 경우를 묶어서 major_cat으로 정리
- summary
    - 있으면 1, 없으면 0으로 변환
- isbn_area
    - 참고 : https://en.wikipedia.org/wiki/List_of_ISBN_registration_groups
    - isbn의 접두사 부분을 파싱해 책이 출판된 국가/지역/언어권 구분
- book_title, img_url, img_path, language, category
    - 삭제
- (선택) 데이터가 적은 class는 others로 통합
    - author, publisher, major_cat, isbn_area
    - 시간이 오래 걸림

In [11]:
def isbn_area(isbn) :
    if isbn[0] in ('0', '1') :
        return '1'
    if isbn[0] in ('2', '3', '4', '5', '7') :
        return isbn[0]
    # 6으로 시작하는 경우 없음
    if isbn[0] == '8' :
        return isbn[:2]
    if isbn[0] == '9' :
        if int(isbn[:2]) < 95 :
            return isbn[:2]
        if int(isbn[:2]) < 99 :
            return isbn[:3]
        else :
            return isbn[:4]
    else :
        return 'others'

def books_ratings_preprocess(raw_books, raw_train_ratings, raw_test_ratings) :
    books = raw_books.copy()
    train_ratings = raw_train_ratings.merge(raw_books[['isbn', 'img_url']], how='left', on='isbn')
    test_ratings = raw_test_ratings.merge(raw_books[['isbn', 'img_url']], how='left', on='isbn')
    
    # isbn
    train_ratings['isbn'] = train_ratings['img_url'].apply(lambda x: x.split('P/')[1][:10])
    test_ratings['isbn'] = test_ratings['img_url'].apply(lambda x: x.split('P/')[1][:10])
    books['isbn'] = books['img_url'].apply(lambda x: x.split('P/')[1][:10])
    
    # book_author
    books['book_author'] = books['book_author'].str\
                        .lower().replace('[^0-9a-zA-Z]', '', regex=True)
    
    # year_of_publication
    bins = [0, 1950, 1960, 1970, 1980, 1990, 2000, 2010]
    books['year_of_publication'] = pd.cut(x=books['year_of_publication'],
                                          bins=bins, right=False, labels=range(7))
    
    # publisher
    books['publisher'] = books['publisher'].str\
                        .lower().replace('[^0-9a-zA-Z]', '', regex=True)
    
    # category
    books['category'] = books['category'].str\
                        .lower().replace('[^0-9a-zA-Z]', '', regex=True)
    author_cat_map = dict(books.groupby('book_author')['category']
                      .value_counts().sort_values().index.tolist())
    books['category'] = books['book_author'].map(author_cat_map)
    publisher_cat_map = dict(books.groupby('publisher')['category']
                      .value_counts().sort_values().index.tolist())
    books['category'] = books['category'].fillna(
                        books['publisher'].map(publisher_cat_map))
    books['category'].fillna('na', inplace=True)
    major_cat = ['fiction', 'juvenilefiction', 'juvenilenonfiction', 'biography',
            'histor', 'religio', 'science', 'social', 'politic', 'humor',
            'spirit', 'business', 'cook', 'health', 'famil', 'computer',
            'travel', 'self', 'poet', 'language', 'art', 'language art',
            'literary', 'criticism', 'nature', 'philosoph', 'reference', 'drama',
            'sport', 'transportation', 'comic', 'craft', 'education', 'crime',
            'music', 'animal', 'garden', 'detective', 'house', 'tech', 'photograph',
            'adventure', 'game', 'architect', 'law', 'antique', 'friend',
            'sciencefiction', 'fantasy', 'mathematic', 'design', 'actor',
            'horror', 'adultery']
    books['major_cat'] = books['category'].copy()
    for category in major_cat :
        books['major_cat'] = np.where(books['category'].str.contains(category),
                                     category, books['major_cat'])
        
    # summary
    books['summary'] = np.where(books['summary'].notnull(), 1, 0)
    
    # isbn_area
    books['isbn_area'] = books['isbn'].apply(isbn_area)
#     # 선택
#     aut_cnt = books['book_author'].value_counts()
#     low_cnt_aut = aut_cnt[aut_cnt < 10].index.tolist()
#     for aut in low_cnt_aut :
#         books['book_author'] = np.where(books['book_author']==aut,
#                                      'others', books['book_author'])
#     # 선택
#     pub_cnt = books['publisher'].value_counts()
#     low_cnt_pub = pub_cnt[pub_cnt < 10].index.tolist()
#     for pub in low_cnt_pub :
#         books['publisher'] = np.where(books['publisher']==pub,
#                                      'others', books['publisher'])
    # 선택
    cat_cnt = books['major_cat'].value_counts()
    low_cnt_cat = cat_cnt[cat_cnt < 10].index.tolist()
    for cat in low_cnt_cat :
        books['major_cat'] = np.where(books['major_cat']==cat,
                                     'others', books['major_cat'])
#     # 선택
#     area_cnt = books['isbn_area'].value_counts()
#     low_cnt_area = area_cnt[area_cnt < 10].index.tolist()
#     for area in low_cnt_cat :
#         books['isbn_area'] = np.where(books['isbn_area']==area,
#                                      'others', books['isbn_area'])
    
    train_ratings.drop(columns=['img_url'], inplace=True)
    test_ratings.drop(columns=['img_url'], inplace=True)
    books.drop(columns=['book_title', 'img_url', 'language', 'category', 'img_path'],
               inplace=True)
    
    return books, train_ratings, test_ratings
    

In [12]:
books, train_ratings, test_ratings = books_ratings_preprocess(raw_books, raw_train_ratings, raw_test_ratings)

In [13]:
books.head()

,isbn,book_author,year_of_publication,publisher,summary,major_cat,isbn_area
0,0002005018,richardbrucewright,6,harperflamingocanada,1,others,1
1,0060973129,carlodeste,5,harperperennial,1,biography,1
2,0374157065,ginabarikolata,5,farrarstrausgiroux,1,medical,1
3,0399135782,amytan,5,putnampubgroup,1,fiction,1
4,0425176428,robertcowley,6,berkleypublishinggroup,1,histor,1


In [20]:
train_ratings.head()

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9


In [21]:
test_ratings.head()

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0


In [27]:
users.to_csv('users.csv', index=False)
books.to_csv('books.csv', index=False)
train_ratings.to_csv('train_ratings.csv', index=False)
test_ratings.to_csv('test_ratings.csv', index=False)
test_ratings.to_csv('sample_submission.csv', index=False)

In [23]:
context_df = train_ratings.merge(users, on='user_id', how='left')\
                    .merge(books, on='isbn', how='left')
context_df.sample(5)

,user_id,isbn,rating,location,age_bin,book_author,year_of_publication,publisher,summary,major_cat,isbn_area
149804,194646,0441822134,8,unitedkingdom,5,alisonuttley,4,acebooks,1,fantasy,1
128745,93047,006051258X,5,newhampshire,5,billscheft,6,perennial,1,fiction,1
284527,173111,1574533371,9,southcarolina,3,conniemayfowler,6,audioliterature,0,fiction,1
112225,8362,055321490X,5,california,2,virginiawoolf,5,bantam,0,fiction,1
221518,51883,0373226101,10,georgia,3,simonwebb,6,harlequin,1,fiction,1
